In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

### read the raw data

In [2]:
data = pd.read_csv('/Users/aaron/repos/visualizedata/ml/definitions/definitions.csv')

# create a boolean indicator =True if the definition is of "machine learning"
data['ml'] = data['term'] == 'machine learning'

print(data.dtypes)
print(data.head(10))

term          object
definition    object
ml              bool
dtype: object
               term                                         definition    ml
0  machine learning  Automatic or semi-automatic updating of equati...  True
1  machine learning  A method to train a machine(any kind) in a hum...  True
2  machine learning  Pattern recognition through a set of constantl...  True
3  machine learning  Machine learning is a predictive computational...  True
4  machine learning  Machine learning is the development of functio...  True
5  machine learning  Machine learning is the application of algorit...  True
6  machine learning  A process where algorithms, paired with data, ...  True
7  machine learning                     Algorithms that can be trained  True
8  machine learning  Machine Learning describes algorithms that can...  True
9  machine learning             Introducing an algorithm to a machine   True


### <span style="color:red">create feature set X (matrix) and vector of labels L</span>

Use [feature extraction methods in scikit-learn](http://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction) to *vectorize* the text of machine learning/AI definitions into an X matrix. 

In [3]:
from sklearn.feature_extraction.text import CountVectorizer

L = data["ml"] # labels
corpus = data['definition'] # corpus of reviews in words
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus)
print(X.shape)

(164, 774)


### let's look at the words in the corpus of "definition"

In [4]:
print(len(vectorizer.vocabulary_))
print("* * * * *")
print(vectorizer.vocabulary_)

774
* * * * *
{'automatic': 60, 'or': 456, 'semi': 577, 'updating': 729, 'of': 448, 'equations': 219, 'parameters': 473, 'using': 734, 'algorithms': 25, 'automated': 59, 'parameter': 472, 'curve': 155, 'response': 557, 'surface': 648, 'fitting': 255, 'method': 409, 'to': 699, 'train': 704, 'machine': 385, 'any': 38, 'kind': 350, 'in': 319, 'human': 306, 'manner': 395, 'achieve': 8, 'the': 676, 'goal': 275, 'copying': 142, 'behavior': 74, 'even': 223, 'better': 81, 'than': 674, 'regular': 548, 'being': 77, 'pattern': 477, 'recognition': 540, 'through': 695, 'set': 586, 'constantly': 139, 'modifying': 421, 'rules': 563, 'learning': 367, 'is': 346, 'predictive': 499, 'computational': 133, 'based': 68, 'on': 450, 'training': 706, 'model': 416, 'ml': 414, 'algorithm': 23, 'data': 158, 'followed': 257, 'by': 92, 'an': 33, 'optimization': 455, 'process': 509, 'whereby': 754, 'tested': 672, 'with': 764, 'test': 671, 'increase': 322, 'accuracy': 7, 'prediction': 497, 'development': 178, 'functi

### <span style="color:red">fit X, L to SVM using gradient descent</span>

[gradient descent documentation](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html)

In [5]:
# fit SVM linear classifier
from sklearn import linear_model
sgd = linear_model.SGDClassifier()
sgd.fit(X, L)

SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='hinge',
              max_iter=1000, n_iter_no_change=5, n_jobs=None, penalty='l2',
              power_t=0.5, random_state=None, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

### assess performance

In [6]:
# look at performance measures
import my_measures

performance_measures = my_measures.BinaryClassificationPerformance(sgd.predict(X), L, 'sgd')
performance_measures.compute_measures()
print(performance_measures.performance_measures)

{'Pos': 54, 'Neg': 110, 'TP': 54, 'TN': 110, 'FP': 0, 'FN': 0, 'Accuracy': 1.0, 'Precision': 1.0, 'Recall': 1.0, 'desc': 'sgd'}


![](https://upload.wikimedia.org/wikipedia/commons/thumb/2/26/Precisionrecall.svg/350px-Precisionrecall.svg.png)

[source](https://en.wikipedia.org/wiki/Precision_and_recall)

### test model on unseen definitions

In [7]:
# machine learning definitions
# Wikipedia, Expert System, Tech Emergence
ml_defs = ["Machine learning is a field of computer science that gives computers the ability to learn without being explicitly programmed.",
          "Machine learning is an application of artificial intelligence (AI) that provides systems the ability to automatically learn and improve from experience without being explicitly programmed. Machine learning focuses on the development of computer programs that can access data and use it learn for themselves.",
          "Machine Learning is the science of getting computers to learn and act like humans do, and improve their learning over time in autonomous fashion, by feeding them data and information in the form of observations and real-world interactions."]

for d in ml_defs:
    print(d)
    print("* * *")

Machine learning is a field of computer science that gives computers the ability to learn without being explicitly programmed.
* * *
Machine learning is an application of artificial intelligence (AI) that provides systems the ability to automatically learn and improve from experience without being explicitly programmed. Machine learning focuses on the development of computer programs that can access data and use it learn for themselves.
* * *
Machine Learning is the science of getting computers to learn and act like humans do, and improve their learning over time in autonomous fashion, by feeding them data and information in the form of observations and real-world interactions.
* * *


In [8]:
# AI definitions
# Wikipedia, Oxford dictionary
ai_defs = ["Artificial intelligence (AI, also machine intelligence, MI) is intelligence demonstrated by machines, in contrast to the natural intelligence (NI) displayed by humans and other animals.",
          "the theory and development of computer systems able to perform tasks that normally require human intelligence, such as visual perception, speech recognition, decision-making, and translation between languages"]

for d in ai_defs:
    print(d)
    print("* * *")

Artificial intelligence (AI, also machine intelligence, MI) is intelligence demonstrated by machines, in contrast to the natural intelligence (NI) displayed by humans and other animals.
* * *
the theory and development of computer systems able to perform tasks that normally require human intelligence, such as visual perception, speech recognition, decision-making, and translation between languages
* * *


In [9]:
# Definitions of unrelated things: kitten, piano, widget
other_defs = ["A kitten, also known as a kitty or kitty cat, is a juvenile cat.",
             "The piano is an acoustic, stringed musical instrument invented in Italy by Bartolomeo Cristofori around the year 1700 in which the strings are struck by hammers.",
             "a small gadget or mechanical device, especially one whose name is unknown or unspecified"]

for d in other_defs:
    print(d)
    print("* * *")

A kitten, also known as a kitty or kitty cat, is a juvenile cat.
* * *
The piano is an acoustic, stringed musical instrument invented in Italy by Bartolomeo Cristofori around the year 1700 in which the strings are struck by hammers.
* * *
a small gadget or mechanical device, especially one whose name is unknown or unspecified
* * *


### function to transform new definitions to a X vector

In [10]:
def get_prediction(definition):
    text_x = vectorizer.transform([definition]).toarray()
    return(sgd.predict(text_x))

### view predicted classifications of new definitions

In [11]:
print("Model predictions for 'machine learning' definitions:")
for mld in ml_defs:
    print(get_prediction(mld))
    
print("* * *")
print("Model predictions for 'AI' definitions:")
for aid in ai_defs:
    print(get_prediction(aid))

print("* * *")
print("Model predictions for other definitions (kitten, piano, widget):")
for otherd in other_defs:
    print(get_prediction(otherd))

Model predictions for 'machine learning' definitions:
[False]
[False]
[ True]
* * *
Model predictions for 'AI' definitions:
[False]
[False]
* * *
Model predictions for other definitions (kitten, piano, widget):
[False]
[ True]
[False]
